In [1]:
import json
import requests
from src import utils
import time
import os
from pprint import pprint

In [2]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api.json'

In [3]:
api = utils.read_json_file(api_path)

In [4]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'
# in_path = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-train.jsonl'

In [5]:
zs_cot = '/userhomes/philhoon/kt-ai-challenge/result/zero-shot-cot-public_mwp_data_v2_preprocess'
zs = '/userhomes/philhoon/kt-ai-challenge/result/zero-shot-public_mwp_data_v2_preprocess'
zs_cot_data = utils.read_jsonlines(zs_cot)
zs_data = utils.read_jsonlines(zs)

In [6]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [7]:
os.makedirs(output_path, exist_ok=True)
output_file_name = 'result-' + input_file.split('/')[-1]
output_file = output_path + '/' + output_file_name
print(f'output_file : {output_file}')

output_file : /userhomes/philhoon/kt-ai-challenge/result/result-openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl


In [8]:
# jsonl_writer = utils.JSONLWriter(output_file)

In [9]:
data = utils.read_jsonlines(input_file)

In [10]:
# file_name

In [11]:
len(data)

1319

In [12]:
# api

In [13]:
# url = api['url']

In [14]:
# headers = api['headers']

In [15]:
# question = "질문 : 파운드 스털링은 가치가 있나요? 용어설명 :  사실목록 : ['1파운드는 법정 화폐입니다.', '법정 화폐는 정부 법령에 의해 뒷받침되며 내재적 가치가 없습니다', '2020년 5월 기준 1파운드 스털링은 약 1.24달러의 가치가 있습니다.']. 사실목록을 바탕으로 질문에 대한 답변을 사실 또는 거짓으로 대답하세요"/

In [16]:
# headers

In [17]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [18]:
model = model_api(api['url'], api['headers'])

In [19]:
BODY_TEMP = {
    "serviceInstanceId" : "ddm3kz9g", 
    "nluType" : "009",
    "apiType" : "002", 
    "utterance" : None
}

In [20]:
# body = json.dumps(body_temp)

In [21]:
# print(body)

In [22]:
# response = api.generate(body)

In [23]:
# context = response.json()['data'][0]['result'][0]

In [24]:
# print(context)

In [25]:
def get_gsm8k_zs_cot(que):
    prompt = f"question : {que} \nAnswer : Let's think step by step."
    return prompt

In [26]:
def get_em_gsm8k_zs(que):
    prompt = f'question : {que} \nAnswer : '
    return prompt

In [27]:
def get_gsm8k_zs(que):
    prompt = f'질문 : {que} \n정답 : '
    return prompt

In [28]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [29]:
# from pprint import pprint

In [30]:
# zs_cot_data 
# zs_data

In [31]:

question = "Jiyoung's class friends are standing in a line in ascending order of height. Jiyoung is standing in the 6th position from the front, and there are a total of 18 students in Jiyoung's class. If they line up again in descending order of height, at what position from the front will Jiyoung be?"
q2 = "Trying to evenly distribute 6 pencils into 3 pencil cases. How many pencils should be put in each case?"
q3 = "In a forest, the animals had a 100m running race, and the tiger finished in 15 seconds, the deer in 20 seconds, the cheetah in 18 seconds, and the wolf in 21 seconds. What was the record of the second fastest animal?"
q4 = "Jungwon spent money as follows during one month: 40% on snacks, 30% on school supplies, 20% on books, which amounted to 10,000 won, and 10% on other items, which amounted to 5,000 won. Calculate the amount spent on snacks."

In [32]:
def get_gsm8k_zs2(que):
    prompt = f'질문: {que}'
    return prompt

In [33]:
def get_gsm8k_zs3(que):
    prompt = f'다음 질문에 차근차근 답하시오. {que}'
    return prompt

In [34]:
def get_gsm8k_zs4(que):
    prompt = f'다음 질문에 답하시오. {que}'
    return prompt

In [35]:
data[1]

{'question': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?',
 'answer': 'It takes 2/2=<<2/2=1>>1 bolt of white fiber\nSo the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric\n#### 3',
 'kor_question': '한 벌의 로브를 만드는 데 파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요합니다. 총 몇 덩이의 섬유가 필요한가요?',
 'kor_answer': '하얀 섬유로 만든 볼트 1개가 필요합니다.\n그러므로 전체 원단 양은 2+1=3개의 볼트입니다.',
 'answer_s': '3'}

In [36]:
cnt = 0
split_cnt = 0
for ins in data:
#     print(f"question :\n{ins['question']}")
#     print(f"zs_infer :\n{ins['zs_infer']}")

#     print("===========================")


    kor_ans = ins['kor_question']
    prompt = get_gsm8k_zs4(kor_ans)
#     print(prompt)
#     break
    
    body = get_body(prompt, BODY_TEMP)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    print('--------')
    print(f"answer_s :\n{ins['answer_s']}")
    print('========')

#     ins['kor_infer'] = res_context
# #     jsonl_writer.write_json_line(ins)
    
#     print(prompt)
    if cnt == 10:
        break
    else:
        cnt += 1

/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
다음 질문에 답하시오. 제넷의 오리는 하루에 16개의 알을 낳습니다. 그녀는 매일 아침 3개를 먹고, 매일 4개를 사용하여 친구들을 위해 머핀을 굽습니다. 그녀는 남은 알을 매일 농부 시장에서 1개당 $2에 판매합니다. 그녀는 매일 농부 시장에서 얼마나 벌까요? 

Answer: 그녀는 매일 농부 시장에서 $16에서 $12를 뺀 $4를 벌게 됩니다.
res_context : 
제넷의 오리는 하루에 16개의 알을 낳습니다. 따라서, 그녀는 매일 농부 시장에서 16 * 12 = $32를 벌게 됩니다.
--------
answer_s :
18


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
다음 질문에 답하시오. 한 벌의 로브를 만드는 데 파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요합니다. 총 몇 덩이의 섬유가 필요한가요?
res_context : 
파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요하므로 총 4 덩이의 섬유가 필요합니다.
--------
answer_s :
3


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

prompt : 
다음 질문에 답하시오. 존은 60mph의 속도로 3시간 동안 운전을 하다가 집에서 매우 중요한 것을 깜빡했다는 것을 깨달았습니다. 그는 4시간 안에 집에 도착하려고 노력하지만 처음 2시간 동안 정체 교통에 갇혀 있었습니다. 그는 그 다음 30분 동안 30mph의 속도로 운전한 후 나머지 4시간 동안 80mph의 속도로 운전할 수 있었습니다. 그가 4시간 후에 집으로부터 얼마나 떨어져 있나요?
res_context : 
존은 4시간 후에 집으로부터 10m 떨어진 곳에 있습니다.
--------
answer_s :
45


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
다음 질문에 답하시오. 엘리자의 시간당 급여는 일주일에 처음 40시간 동안은 10달러입니다. 그녀는 또한 근무 시간 외에 기본 시간당 급여의 1.2배를 받습니다. 이번 주에 엘리자가 45시간 일했다면, 이번 주 그녀의 수입은 얼마입니까?
res_context : 
엘리자가 일주일에 처음 40시간 동안은 10달러이므로, 시간당 급여는 10/40 = 2.5달러입니다. 그녀는 근무 시간 외에 기본 시간당 급여의 1.2배를 받으므로, 기본 시간당 급여는 2.5/40 x 1.2 = 3.25달러입니다. 엘리자가 이번 주에 45시간 일했다면, 이번 주 그녀의 수입은 3.25달러 + (1.2달러 x 45시간) = 6달러입니다.
--------
answer_s :
460


/userhomes/philhoon/miniconda3/envs/kt/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
다음 질문에 답하시오. 한 달 동안 새로운 프로그램은 60번 다운로드되었습니다. 두 번째 달의 다운로드 횟수는 첫 번째 달의 다운로드 횟수의 3배였지만, 세 번째 달에는 30% 감소했습니다. 이 프로그램은 총 3개월 동안 얼마나 많이 다운로드되었습니까?
res_context : 
이 프로그램은 총 3개월 동안 60번 다운로드되었습니다.
--------
answer_s :
366


### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [ ]:
cnt = 0
split_cnt = 0
for ins in data:
    kor_que = ins['kor_question']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue

    kor_ans = ins['kor_answer']
    prompt = get_gsm8k_zs(kor_que)
    
    body = get_body(prompt, BODY_TEMP)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    print('----')
    
    ins['kor_infer'] = res_context
#     jsonl_writer.write_json_line(ins)
    
#     print(prompt)
    if cnt == 5:
        break
    else:
        cnt += 1